In [1]:
import os
from datetime import datetime
import time
import pandas as pd
import numpy as np

In [2]:
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}

In [3]:
data_dir = '/opt/ml/input/data/train_dataset/'

### 사용하려는 dataset 이름을 아래 cell 에서 설정해주세요!!!!

In [4]:
# Baseline dataset 사용하는 경우 아래 주석 해제!
train_name = 'train_data.csv'
test_name = 'test_data.csv'

# Time delta dataset 사용하는 경우 아래 주석 해제!
# train_name = 'train_all_three.csv'
# test_name = 'test_all_three.csv'

In [5]:
raw_train_df = pd.read_csv(os.path.join(data_dir, train_name), dtype=dtype, parse_dates=['Timestamp'])
raw_train_df = raw_train_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

raw_test_df = pd.read_csv(os.path.join(data_dir, test_name), dtype=dtype, parse_dates=['Timestamp'])
raw_test_df = raw_test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

# TODO: 전체 데이터에 대해서 통계량 계산 가능
# full_df = pd.concat([raw_train_df, raw_test_df], ignore_index=True)
# full_df = full_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

### train 데이터에서 시험지/문제/태그별 평균 정답률과 총 정답 수 뽑기

In [6]:
testId_mean_sum = raw_train_df.groupby(['testId'])['answerCode'].agg(['mean','sum']).to_dict()
assessmentItemID_mean_sum = raw_train_df.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum']).to_dict()
KnowledgeTag_mean_sum = raw_train_df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum']).to_dict()

### train 데이터에서 대분류별 평균 정답률과 총 정답 수 뽑기

In [7]:
raw_train_df['assessment_category'] = raw_train_df.apply(lambda row: row.assessmentItemID[2], axis=1)

assessment_category_mean_sum = raw_train_df.groupby(['assessment_category'])['answerCode'].agg(['mean', 'sum']).to_dict()

### train 데이터에서 문제/태그별 평균 소요 시간 뽑기

In [8]:
# 먼저 소요 시간을 계산

def convert_time(s):
    timestamp = time.mktime(s.timetuple())
    return int(timestamp)

if 'elapsed' in raw_train_df.columns:
    raw_train_df['elapsed_time'] = raw_train_df['elapsed'].apply(lambda x: np.log(x))
else:
    # TODO: Deal with zeros first before using log of elapsed_time
    raw_train_df['Timestamp_int'] = raw_train_df['Timestamp'].apply(convert_time)
    raw_train_df['elapsed_time'] = raw_train_df.loc[:, ['userID', 'Timestamp_int', 'testId']].groupby(
        ['userID', 'testId']).diff().shift(-1).fillna(int(10))

In [9]:
et_by_kt_mean = raw_train_df.groupby(['KnowledgeTag'])['elapsed_time'].agg(['mean']).to_dict()
et_by_as_mean = raw_train_df.groupby(['assessmentItemID'])['elapsed_time'].agg(['mean']).to_dict()

### 구한 train 통계량들로 데이터 만들기

In [10]:
def set_basic_stats(df):
    df["testId_answer_rate"] = df.testId.map(testId_mean_sum['mean'])
    df['testId_answer_sum'] = df.testId.map(testId_mean_sum['sum'])
    df["assessmentItemID_answer_rate"] = df.assessmentItemID.map(assessmentItemID_mean_sum['mean'])
    df['assessmentItemID_answer_sum'] = df.assessmentItemID.map(assessmentItemID_mean_sum['sum'])
    df["knowledge_tag_mean"] = df.KnowledgeTag.map(KnowledgeTag_mean_sum['mean'])
    df['knowledge_tag_sum'] = df.KnowledgeTag.map(KnowledgeTag_mean_sum['sum'])

    if 'assessment_category' not in df.columns:
        df['assessment_category'] = df.apply(lambda row: row.assessmentItemID[2], axis=1)
    df['assessment_category_mean'] = df.assessment_category.map(assessment_category_mean_sum['mean'])

    if 'elapsed_time' not in df.columns:
        if 'elapsed' in df.columns:
            df['elapsed_time'] = df['elapsed'].apply(lambda x: np.log(x))
        else:
            df['Timestamp_int'] = df['Timestamp'].apply(convert_time)
            df['elapsed_time'] = df.loc[:, ['userID', 'Timestamp_int', 'testId']].groupby(
                ['userID', 'testId']).diff().shift(-1).fillna(int(10))

    df['et_by_kt'] = df.KnowledgeTag.map(et_by_kt_mean['mean'])
    df['et_by_as'] = df.assessmentItemID.map(et_by_as_mean['mean'])

    return df

In [11]:
raw_train_df = set_basic_stats(raw_train_df)
raw_train_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,assessment_category,Timestamp_int,elapsed_time,testId_answer_rate,testId_answer_sum,assessmentItemID_answer_rate,assessmentItemID_answer_sum,knowledge_tag_mean,knowledge_tag_sum,assessment_category_mean,et_by_kt,et_by_as
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,6,1585009031,3.0,0.947683,1268.0,0.982063,219.0,0.955022,637.0,0.709232,20.505247,13.762332
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,6,1585009034,8.0,0.947683,1268.0,0.964126,215.0,0.913187,3040.0,0.709232,11339.147492,26.708520
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,6,1585009042,7.0,0.947683,1268.0,0.910314,203.0,0.913187,3040.0,0.709232,11339.147492,19.237668
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,6,1585009049,7.0,0.947683,1268.0,0.968610,216.0,0.913187,3040.0,0.709232,11339.147492,18.811659
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,6,1585009056,11.0,0.947683,1268.0,0.941704,210.0,0.913187,3040.0,0.709232,11339.147492,34.529148


In [12]:
raw_test_df = set_basic_stats(raw_test_df)
raw_test_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,testId_answer_rate,testId_answer_sum,assessmentItemID_answer_rate,assessmentItemID_answer_sum,knowledge_tag_mean,knowledge_tag_sum,assessment_category,assessment_category_mean,Timestamp_int,elapsed_time,et_by_kt,et_by_as
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,0.560944,856.0,0.646789,141.0,0.641379,1023.0,5,0.658208,1578567391,26.0,103.047022,150.408257
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,0.560944,856.0,0.628440,137.0,0.641379,1023.0,5,0.658208,1578567417,94.0,103.047022,112.472477
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,0.560944,856.0,0.577982,126.0,0.670013,1535.0,5,0.658208,1578567511,5.0,122.663029,59.348624
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,0.560944,856.0,0.655963,143.0,0.670013,1535.0,5,0.658208,1578567516,7.0,122.663029,151.344037
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,0.560944,856.0,0.307339,67.0,0.568970,2314.0,5,0.658208,1578567523,3.0,20855.813622,74.316514


### 새로운 데이터를 csv 로 저장
- 원래 데이터 이름 뒤에 "basic_stats" 가 붙은 새로운 파일로 저장됩니다.

In [13]:
train_name = f"{train_name.split('.')[0]}_basic_stats.csv"
test_name = f"{test_name.split('.')[0]}_basic_stats.csv"

raw_train_df.to_csv(os.path.join(data_dir, train_name), sep=',', index=False)
raw_test_df.to_csv(os.path.join(data_dir, test_name), sep=',', index=False)